In [ ]:
!pip install -q transformers datasets peft bitsandbytes accelerate trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.6/564.6 kB 32.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset_name = "lavita/AlpaCare-MedInstruct-52k"
dataset = load_dataset(dataset_name, split="train")

print(dataset[0])

README.md:   0%|          | 0.00/944 [00:00<?, ?B/s]

data/train-00000-of-00001-297892d5d4e8a0(…):   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

{'output': 'A mass in the lung could cause shortness of breath due to several reasons. First, the mass can physically obstruct the air passages, causing difficulty in airflow and leading to breathing difficulties. Second, if the mass is cancerous or infected, it can cause inflammation and damage to lung tissue, reducing its functional capacity and compromising normal breathing. Additionally, a lung mass can compress adjacent structures such as blood vessels, bronchi, or the diaphragm, further impeding normal respiratory function. Overall, any interference with the normal flow of air in the lungs caused by a mass can result in inadequate oxygen exchange and subsequent shortness of breath.\n\nThe answer is: A mass in the lung can obstruct air passages, cause inflammation, damage lung tissue, and compress adjacent structures, leading to shortness of breath.', 'input': '<noinput>', 'instruction': 'Explain why a mass in the lung could cause shortness of breath.'}


In [ ]:
# This function creates a standardized prompt template
def format_prompt(example):
    # The 'input' field is often empty, handle that case
    if example.get("input"):
        return f"""### Instruction:
{example['instruction']}

### Input:
{example['input']}

### Response:
{example['output']}"""
    else:
        return f"""### Instruction:
{example['instruction']}

### Response:
{example['output']}"""

# Example of applying it (you'll use this in the trainer later)
print(format_prompt(dataset[0]))

### Instruction:
Explain why a mass in the lung could cause shortness of breath.

### Input:
<noinput>

### Response:
A mass in the lung could cause shortness of breath due to several reasons. First, the mass can physically obstruct the air passages, causing difficulty in airflow and leading to breathing difficulties. Second, if the mass is cancerous or infected, it can cause inflammation and damage to lung tissue, reducing its functional capacity and compromising normal breathing. Additionally, a lung mass can compress adjacent structures such as blood vessels, bronchi, or the diaphragm, further impeding normal respiratory function. Overall, any interference with the normal flow of air in the lungs caused by a mass can result in inadequate oxygen exchange and subsequent shortness of breath.

The answer is: A mass in the lung can obstruct air passages, cause inflammation, damage lung tissue, and compress adjacent structures, leading to shortness of breath.


In [ ]:
# Shuffle the dataset
shuffled_dataset = dataset.shuffle(seed=42)

# Split 90% for training
train_test_split = shuffled_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']

# Split the remaining 10% into 5% validation and 5% test
test_val_split = train_test_split['test'].train_test_split(test_size=0.5)
validation_dataset = test_val_split['train']
test_dataset = test_val_split['test']

print(f"Train size: {len(train_dataset)}")
print(f"Validation size: {len(validation_dataset)}")
print(f"Test size: {len(test_dataset)}")

Train size: 46801
Validation size: 2600
Test size: 2601


In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': " I am a large language model developed by Mistral AI. I am designed to generate human-like text based on the input I receive. I don't have the ability to have experiences, emotions or physical sensations, but I can process text and generate responses based on the data I've been trained on. I'm here to help answer questions, generate creative content, and assist with various tasks to the best of my ability."}]}]

In [ ]:
from huggingface_hub import login
from google.colab import userdata

try:
    hf_token = userdata.get('HF_TOKEN')
    login(token=hf_token)
    print("Successfully logged in to Hugging Face.")
except Exception as e:
    print(f"Could not log in to Hugging Face. Please ensure you have added your token to Colab secrets named 'HF_TOKEN' and enabled Notebook access.")
    print(f"Error: {e}")

Successfully logged in to Hugging Face.


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# 4-bit quantization config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    llm_int8_enable_fp32_cpu_offload=True # Enable CPU offloading
)

# Load the base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto", # Automatically uses the GPU, may still offload to CPU with above config
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token # Set padding token

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig

lora_config = LoraConfig(
    r=16,  # Rank of the update matrices. Higher rank means more parameters to train.
    lora_alpha=32, # Alpha scaling factor.
    target_modules=["q_proj", "v_proj"], # Target the attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=1, # At least 1 epoch as required
    logging_steps=25,
    save_steps=500,
    fp16=True, # Use mixed precision
    max_steps=1000 # For a quicker run, you can use max_steps instead of epochs
)

trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset, # Use your pre-processed training data
    eval_dataset=validation_dataset,
    peft_config=lora_config,
    # dataset_text_field="text", # You'll need to reformat your dataset to have a single 'text' field using the format_prompt function - Removed as formatting_func is used
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_args,
    formatting_func=format_prompt, # Use the formatting function directly
)

# Start training
trainer.train()